In [ ]:
from yolo import *
import os
import pandas as pd

CURR_DIR = '/home/misho/Uni/Vision/YOLO/keras-YOLOv3-model-set'

### Initialize classification function

In [ ]:
yolo = YOLO_np(**{'dump_model':False, 'image':True, 'input':'./path2your_video', 'model_image_size':(416, 416), 'output':'', 'pruning_model':False})
print(os.getcwd())

def car_on_image(path):
    img = Image.open(path)

    out_boxes, out_classes, out_scores = yolo.detect_image(img, True) 
    for i in range(len(out_classes)):
        if out_classes[i] == 2:
            box = out_boxes[i]
            box_area = (box[2]-box[0]) * (box[3]-box[1]) / (275*350)
            if box_area > 0.25:
                return True
    return False

### Load dataset

In [ ]:
data = pd.read_csv('./Myauto_data/MyAuto_ge_Cars_Data.csv')
data.head()

### Run classifier and create dataframe for images that contain car for each car ID

In [ ]:
car_image_df = pd.DataFrame({'ID':[], 'images':[]})
path = CURR_DIR + '/Myauto_data/Car_Images/{}/'
for i in range(len(data)):
    ID = data.iloc[i].ID
    images = ''
    try:
        image_dir = os.listdir(path.format(ID))
    except: continue
    for img in image_dir:
        if car_on_image(path.format(ID)+img):
            images += img.split('.')[0] + '_'
    if len(images) != 0:
        images = images[:-1]
    car_image_df = car_image_df.append(pd.DataFrame({'ID':[int(ID)], 'images':[images]}))
    if i%100 == 0:
        car_image_df.to_csv('CAR_IMAGE_DF.csv')
        print('Done: {}'.format(i))